# Outline

## Direct CNOT

The direct CNOT can be obtained using the Hamiltonian from Schrieffer-Wolff perturbation theory. 
\begin{equation}
    H = \Omega_C(t) (g_{IX} IX + g_{ZX} ZX)
\end{equation}
We remark two things: 
1. The control qubit operators commute with Z, while the target qubit operators do not.
2. The relative strength of the coefficients depends on the pair of qubits involved. The unitary trajectory of the gate, and therefore the relative crosstalk contribution, will vary slightly.

### Remark 1

Paying attention to the fact that the control commutes with Z but the target does not, we draw a few additional conclusions. If we are robust to $IZ$ within the gate, then we are in fact robust to $ZZ$. Loosely speaking,
\begin{equation}
    U^\dag(t) Z_C Z_T U(t) = U^\dag(t) I_C Z_T U(t) U^\dag(t) Z_C I_T U(t) = U^\dag(t) I_C Z_T U(t) Z_C I_T
\end{equation}
and only the target term contributes.

The control crosstalk remains unless we use an additional drive to cancel it. We can drive an identity gate on the 

### Remark 2

We can make point 2 clearer if we absorb one of the coefficients into the control drive. Then,
\begin{equation}
    H = \widetilde{\Omega}_C(t) (IX + \mu ZX).
\end{equation}

Of course, we can also drive on the target, so
\begin{equation}
    H = \widetilde{\Omega}_C(t) (IX + \mu ZX) + \Omega_T(t) IX.
\end{equation}
Now we have relative control over the IX rate, so we could optimize for the same unitary dynamics for any gate. The pulse shapes will be different, but the robustness behavior will be what we want.


## Echo CR

Many of the previous insight carry over. Suppose we have the following Hilbert space:
\begin{equation}
CS \leftrightarrow C \leftrightarrow T \leftrightarrow TS.
\end{equation}
The Echo CR is a gate sequence in the control-target subspace given by $XI \circ R_{ZX}(-\frac{\pi}{4}) \circ XI \circ R_{ZX}(\frac{\pi}{4})$,
up to Z rotations on the control or target qubits. Technically, $R_{ZX}$ is a more complicated interaction with other two qubit Hamiltonian terms.

We are focused on ZZ crosstalk. The IZZI crosstalk within the $(C,T)$ subspace vanishes due to the pair of IXII gates. This happens independent of the value of $R_{ZX}(\phi)$.  As such, the ZZII crosstalk in the $(CS, C)$ subspace also vanishes. What remains is the IIZZ crosstalk present in $(T,TS)$. However, we can design the IZXI interaction to account for this crosstalk, and coordinate the ZX and XI pulses to preserve the cancellation of the IZZI crosstalk (if this is even necessary--perhaps we just make the $R_{ZX}(\phi)$ robust and utilize our robust $X$ gates to prevent $ZZ$ accumulation at each step, separately.

## Generalzing the echo
Notice that we can perform the echo CR optimization in a direct CNOT, also. Alternatively, we can apply DD on all target-specator qubits that are not involved in another two qubit gate. (recall that two Echo CR gates might be applied next to each other, $(C_1, T_1, C_2, T_2)$.


In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using CairoMakie
using DelimitedFiles
using Distributions
using LinearAlgebra

In [ ]:
# Operators 
const n_levels = 2
at = create(n_levels)
a = annihilate(n_levels)

H_operators = Dict(
        "X" => a + at,
        "Y" => -im * (a - at),
        "Z" => I - 2 * at * a,
)

single_system = QuantumSystem(zeros(2, 2), [H_operators["X"]])

In [ ]:
## One qubit rotation

In [ ]:
# Load up the single qubit X gate.
# [X, X, SX, SX]
single_robust = load_traj("saved-pulses-2023-12-13/single_qubit_gateset_R1e-3.jld2");
single_default = load_traj("saved-pulses-2023-12-13/single_qubit_gateset_default.jld2");

fig = Figure(resolution = (800, 600))
axes = [Axis(fig[1, 1], title="Default", xlabel="Time (ns)", ylabel="Amplitude (MHz)"),
        Axis(fig[1, 2], title="Robust", xlabel="Time (ns)", ylabel="Amplitude (MHz)")]
data = [single_default[:a], single_robust[:a]]
four_colors = [:red, :blue, :green, :orange]
for (controls, ax) in zip(data, axes)
    for (i, row) in enumerate(eachrow(controls))
        lines!(ax, row, color = four_colors[i], linewidth = 2, linestyle = :solid)
    end
end
fig

In [ ]:
function rollout_final(controls, system, Δt=2/9)
    return iso_vec_to_operator(unitary_rollout(controls, Δt, system)[:,end])
end

In [ ]:
rollout_final(single_robust[:a][1:1, :], single_system)

In [ ]:
rollout_final(single_default[:a][1:1, :], single_system)

## Two qubit rotation

In [ ]:
H_crosstalk = (
    kron_from_dict("IZ", H_operators)
    + kron_from_dict("ZZ", H_operators)
)

@views function infidelity_robustness(Hₑ::AbstractMatrix, p::QuantumControlProblem)
    Z⃗ = vec(p.trajectory.data)
    Z = p.trajectory
    return InfidelityRobustnessObjective(Hₑ, Z).L(Z⃗, Z)
end

Pretend the units are now in MHz and us.

Our rotation gates are approximately 200 ns.

In [ ]:
# Time: 92.2 ns π/4 gate
# Adjust units by factor of 10 (ns * 10, GHz / 10)
T = 84
Δt = 1/9
println(T * Δt)
;

H_drift2= zeros(4, 4)
H_controls2 = [kron_from_dict("ZX", H_operators)]
double_system = QuantumSystem(H_drift2, H_controls2)
U_goal = exp(-im * kron_from_dict("ZX", H_operators) * π / 4)
;

In [ ]:
a_bound = 0.10

prob = UnitarySmoothPulseProblem(
    H_drift2,
    H_controls2,
    U_goal,
    T,
    Δt;
    a_bound=a_bound,
    dda_bound=0.1 * a_bound,
    a_guess=(π/4/(T * Δt)) * ones((length(H_controls2), T)),
    timesteps_all_equal=true,
    free_time=true,
    hessian_approximation=true,
    pade_order=20,
    # R_a=1e-9,
    # R_da=10.0,
    # R_dda=1e5,
)

In [ ]:
# Reasonable control coupling is ~1 MHz in frequency units without 2π 
2 * π * .005 # GHz

In [ ]:
# The value we need to get the rotation we want (in the original GHz units)
π/4/(T * Δt)/10

In [ ]:
# Tranlating a_bound to GHz
a_bound / 10

In [ ]:
solve!(prob; max_iter=200)

println("Fidelity: ", unitary_fidelity(prob))
println("Robustness: ", infidelity_robustness(H_crosstalk, prob))

In [ ]:
plot(prob.trajectory)

In [ ]:
print("Max controls: ", maximum(abs.(prob.trajectory[:a])))

In [ ]:
save("saved-pulses/two_qubit_default.jld2", prob.trajectory)
writedlm("saved-pulses/a_two_qubit_default.csv", prob.trajectory[:a], ",")

# Robust solve

In [ ]:
function random_a_guess(traj::NamedTrajectory)
    # Positive (symmetric) upper bounds
    a_bounds = traj.bounds[:a][2]

    a_dists = [Uniform(
        -(a_bounds[i] == Inf ? 1.0 : a_bounds[i]),
        (a_bounds[i] == Inf ? 1.0 : a_bounds[i])
    ) for i = 1:traj.dims[:a]]

    a = hcat([
        zeros(traj.dims[:a]),
        vcat([rand(a_dists[i], 1, traj.T - 2) for i = 1:traj.dims[:a]]...),
        zeros(traj.dims[:a])
    ]...)
    return a
end

In [ ]:
trajectory = copy(prob.trajectory)
parameters = deepcopy(prob.params)
update!(trajectory, :a, 0.8 * (trajectory[:a] + 0.2 * random_a_guess(trajectory)))
update!(trajectory, :Ũ⃗, 2 * rand(trajectory.dims[:Ũ⃗], T) .- 1)

objective = DefaultObjective()
objective += Objective(parameters[:objective_terms][1])
objective += QuadraticRegularizer(:dda, trajectory, 1e-5)
objective += QuadraticRegularizer(:a, trajectory, 1e-6)

# Baseline: Robustness is 0.0014
# objective += QuadraticRegularizer(:dda, trajectory, 1e-4)
# objective += QuadraticRegularizer(:a, trajectory, 1e-4)

new_a_bound = 0.5
update_bound!(trajectory, :a, new_a_bound)
update_bound!(trajectory, :dda, 10 * new_a_bound)
constraints = trajectory_constraints(trajectory)

ipopt_options = Options()
ipopt_options.hessian_approximation = "limited-memory"

rob_prob = UnitaryRobustnessProblem(
    H_crosstalk,
    trajectory,
    prob.system,
    objective,
    prob.integrators,
    constraints;
    final_fidelity=0.9999,
    verbose=false,
    build_trajectory_constraints=false,
    hessian_approximation=true,
    ipopt_options=ipopt_options
)

In [ ]:
solve!(rob_prob; max_iter=500)

println("Fidelity: ", unitary_fidelity(rob_prob))
println("Robustness: ", infidelity_robustness(H_crosstalk, rob_prob))

In [ ]:
print("Max controls: ", maximum(abs.(rob_prob.trajectory[:a])))

In [ ]:
plot(rob_prob.trajectory)

In [ ]:
save("saved-pulses/two_qubit_robust.jld2", rob_prob.trajectory)

In [ ]:
writedlm("saved-pulses/a_two_qubit_robust.csv", rob_prob.trajectory[:a], ",")

In [ ]:
infidelity_robustness(kron_from_dict("ZZ", H_operators), rob_prob)

In [ ]:
println("Robust: ", infidelity_robustness(kron_from_dict("IZ", H_operators), rob_prob))
println("Initial: ", infidelity_robustness(kron_from_dict("IZ", H_operators), prob))

In [ ]:
println("Robust: ", infidelity_robustness(kron_from_dict("ZZ", H_operators), rob_prob))
println("Initial: ", infidelity_robustness(kron_from_dict("ZZ", H_operators), prob))

In [ ]:
println("Robust: ", infidelity_robustness(kron_from_dict("ZI", H_operators), rob_prob))
println("Initial: ", infidelity_robustness(kron_from_dict("ZI", H_operators), prob))